 
#<font color='yellow'>***/***</font> ***ProLink*** <font color='yellow'>***/***</font>  

2022 @ *Universidad de Zaragoza*

For more information please visit https://github.com/unizar-flav/ProLink




In [ ]:
#@title Install environment
#@markdown Run this cell first!
%%capture

import os

!wget https://github.com/marbl/Mash/releases/download/v2.3/mash-Linux64-v2.3.tar 
!tar -xvf mash-Linux64-v2.3.tar 

!mv /content/mash-Linux64-v2.3 /content/mashdir
os.environ['PATH'] += ":/content/mashdir"

!wget https://github.com/soedinglab/MMseqs2/releases/download/14-7e284/mmseqs-linux-avx2.tar.gz 
!tar -xzf mmseqs-linux-avx2.tar.gz 
os.environ['PATH'] += ":/content/mmseqs/bin"

!git clone https://github.com/unizar-flav/ALFATClust.git 
os.environ['PATH'] += ":/content/ALFATClust/main"

!wget https://github.com/rcedgar/muscle/releases/download/v5.1/muscle5.1.linux_intel64 
!mv /content/muscle5.1.linux_intel64 /content/muscle
!chmod +x /content/muscle
os.environ['PATH'] += ":/content"

!apt-get update
!apt-get install ghostscript

!wget https://www.megasoftware.net/do_force_download/megax-cc_10.2.6-1_amd64.deb 
!apt-get remove -y --purge man-db 
!dpkg -i megax-cc_10.2.6-1_amd64.deb 
!apt --fix-broken install  

!git clone https://github.com/unizar-flav/ProLink.git

%pip install -r ProLink/requirements.txt

from ProLink.Main import pro_link
from google.colab import files

In [ ]:
#@title Introduce the parameters { run: "auto" }

import configparser

config = configparser.ConfigParser()
config.read("/content/ProLink/parameters.cfg")

parameters = {}
for section in config.sections():
    parameters[section] = {}
    for option in config.options(section):
        parameters[section][option] = config.get(section, option)


#@markdown ----QUERY PROTEINS UNIPROT CODES----
query_proteins = "ABQ62066.1, ABQ62091.1, ABQ62490.1" #@param {type:"string"}

#@markdown ----BLAST PARAMETERS----
hitlist_range = 5000 #@param {type:"number"}
up = {'hitlist_range': hitlist_range}
parameters['Blast'].update(up)

blast_database = "Reference Proteins (refseq_protein)" #@param ["Non redundant protein sequences (nr)", "Reference Proteins (refseq_protein)", "Model organisms (landmark)", "Protein Data Bank proteins (pdb)"]
blast_database = blast_database.split('(')[1].split(')')[0] # database name between parenthesis

up = {'blast_database': blast_database}
parameters['Blast'].update(up)

smart_blast_ = False #@param {type:"boolean"}
up = {'smart_blast_': smart_blast_}
parameters['Blast'].update(up)

#@markdown ----CLUSTERING----
cluster_seqs = True #@param {type:"boolean"}
up = {'cluster_seqs': cluster_seqs}
parameters['Clustering'].update(up)

similarity = 0.5 #@param {type:"number"} #Initial similarity treshold to group the sequences into clusters.
up = {'similarity': similarity}
parameters['Clustering'].update(up)

smart_clustering_ = True #@param {type:"boolean"}
up = {'smart_clustering_': smart_clustering_}
parameters['Clustering'].update(up)

#@markdown ----PFAM DOMAINS----
check_pfam_domains = True #@param {type:"boolean"}
up = {'check_pfam_domains': check_pfam_domains}
parameters['Pfam domains'].update(up)

#@markdown ----ALIGNMENT----
align_seqs = True #@param {type:"boolean"}
up = {'align_seqs': align_seqs}
parameters['Alignment'].update(up)

#@markdown ----SEQUENCE LOGO GENERATION----
generate_logo = True #@param {type:"boolean"}
up = {'generate_logo': generate_logo}
parameters['Weblogo'].update(up)

#@markdown ----PHYLOGENETIC TREE GENERATION----
generate_tree = True #@param {type:"boolean"}
up = {'generate_tree': generate_tree}
parameters['Tree'].update(up)

tree_type = "NJ" #@param ["NJ", "ML"]
up = {'tree_type': tree_type}
parameters['Tree'].update(up)

#@markdown For more advanced options, please feel free to edit the ProLink/parameters.cfg file.

In [ ]:
#@title Execute the script
#@markdown This may take a while

from datetime import datetime

now = str(datetime.now()).replace(" ", "_")
outputs_dir = "outputs_" + str(now)
up = {'outputs_dir': outputs_dir}
parameters['Outputs'].update(up)
print(now)
print(parameters)
download_outputs = True #@param {type:"boolean"}

pro_link(query_proteins, **parameters) 

if download_outputs:
  downloadfile = "/content/" + str(outputs_dir)
  output_filename = 'ProLink_'+str(outputs_dir)+'.zip'
  os.system( "zip -r {} {}".format( output_filename , downloadfile ) )
  files.download( output_filename )

In [ ]:
#@title (Optional)  Download outputs manually { display-mode: "form" }
#@markdown Copy the path of the file/directory below and run the cell to download.
downloadfile = "" #@param {type: "string"}

output_filename = 'ProLink_outputs.zip'
os.system( "zip -r {} {}".format( output_filename , downloadfile ) )
files.download( output_filename )